In [23]:
import random
import heapq
from itertools import count

In [24]:
class Node:
    def __init__(self, data, level, fval):
        self.data = data
        self.level = level
        self.fval = fval

    def generate_child(self):
        """Generate child nodes by moving the blank space in four possible directions."""
        x, y = self.find_blank(self.data, '_')
        directions = [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]
        children = []
        for new_x, new_y in directions:
            child_data = self.shuffle(self.data, x, y, new_x, new_y)
            if child_data:
                children.append(Node(child_data, self.level + 1, 0))
        return children

    def shuffle(self, puzzle, x1, y1, x2, y2):
        """Move the blank space to a specified direction if within bounds."""
        if 0 <= x2 < len(puzzle) and 0 <= y2 < len(puzzle[0]):
            new_puzzle = [row[:] for row in puzzle]  # Deep copy of the puzzle
            new_puzzle[x1][y1], new_puzzle[x2][y2] = new_puzzle[x2][y2], new_puzzle[x1][y1]
            return new_puzzle  # Return as a list of lists
        return None

    def find_blank(self, puzzle, val):
        """Find the position of the blank space."""
        for i, row in enumerate(puzzle):
            if val in row:
                return i, row.index(val)
        return None

class Puzzle:
    def __init__(self, size, goal):
        self.n = size
        self.goal = goal
        self.goal_positions = {val: (i, j) for i, row in enumerate(goal) for j, val in enumerate(row)}
        self.open_list = []
        self.closed_set = set()

    def generate_random_start(self, moves=20):
        """Generate a random initial state starting from the goal state."""
        start = [row[:] for row in self.goal]
        blank_pos = self.find_position(start, '_')

        for _ in range(moves):
            x, y = blank_pos
            directions = [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]
            random.shuffle(directions)
            for new_x, new_y in directions:
                if 0 <= new_x < self.n and 0 <= new_y < self.n:
                    start[x][y], start[new_x][new_y] = start[new_x][new_y], start[x][y]
                    blank_pos = (new_x, new_y)
                    break
        return start  # Return as a list of lists

    def h(self, state):
        """Calculate the Manhattan distance heuristic using memoized goal positions."""
        dist = 0
        for i in range(self.n):
            for j in range(self.n):
                tile = state[i][j]
                if tile != '_' and tile in self.goal_positions:
                    x_goal, y_goal = self.goal_positions[tile]
                    dist += abs(i - x_goal) + abs(j - y_goal)
        return dist

    def find_position(self, puzzle, value):
        """Find the position of a specific value."""
        for i, row in enumerate(puzzle):
            if value in row:
                return i, row.index(value)
        return None

    def f(self, node):
        """Calculate f(x) = g(x) + h(x)."""
        return self.h(node.data) + node.level

    def solve(self, start_state):
        """Solve the puzzle using A* and print only the final solution path."""
        counter = count()
        parent_map = {tuple(map(tuple, start_state)): None}  # Use tuple of tuples to make it hashable
        start_node = Node(start_state, 0, self.f(Node(start_state, 0, 0)))
        heapq.heappush(self.open_list, (start_node.fval, next(counter), start_node))

        goal_node = None

        while self.open_list:
            _, _, current_node = heapq.heappop(self.open_list)

            # Check if it's the goal state
            if self.h(current_node.data) == 0:
                goal_node = current_node
                break

            self.closed_set.add(tuple(map(tuple, current_node.data)))  # Add tuple to set for comparison

            for child in current_node.generate_child():
                child_tuple = tuple(map(tuple, child.data))  # Convert child to tuple for hashing
                if child_tuple not in self.closed_set:
                    child.fval = self.f(child)
                    heapq.heappush(self.open_list, (child.fval, next(counter), child))
                    self.closed_set.add(child_tuple)
                    parent_map[child_tuple] = current_node

        # Print the solution path
        if goal_node:
            self.print_solution(goal_node, parent_map)

    def print_solution(self, node, parent_map):
        """Print the solution path from the initial state to the goal state."""
        path = []
        while node:
            path.append(node.data)
            node = parent_map[tuple(map(tuple, node.data))]

        print("\nSolution path from the initial state to the goal state:\n")
        for step, state in enumerate(reversed(path)):
            print(f"Step {step}:")
            for row in state:
                print(" ".join(row))  # Print the puzzle
            print("")


In [25]:


"""
#4x4 grid

goal_state = [
    ['1', '2', '3', '4'],
    ['5', '6', '7', '8'],
    ['9', '10', '11', '12'],
    ['13', '14', '15', '_']
]
puzzle = Puzzle(4, goal_state)
"""

# Goal state
goal_state = [
    ['1', '2', '3'],
    ['4', '5', '6'],
    ['7', '8', '_']
]
# Initialize the puzzle and generate a random starting state
puzzle = Puzzle(3, goal_state)



start_state = puzzle.generate_random_start(moves=1000)# 1000 random moves

print("\nRandomly generated initial state:")
for row in start_state:
    print(" ".join(row))

# Solve the puzzle
puzzle.solve(start_state)



Randomly generated initial state:
_ 5 8
2 3 1
7 4 6

Solution path from the initial state to the goal state:

Step 0:
_ 5 8
2 3 1
7 4 6

Step 1:
5 _ 8
2 3 1
7 4 6

Step 2:
5 8 _
2 3 1
7 4 6

Step 3:
5 8 1
2 3 _
7 4 6

Step 4:
5 8 1
2 _ 3
7 4 6

Step 5:
5 8 1
2 4 3
7 _ 6

Step 6:
5 8 1
2 4 3
_ 7 6

Step 7:
5 8 1
_ 4 3
2 7 6

Step 8:
5 8 1
4 _ 3
2 7 6

Step 9:
5 _ 1
4 8 3
2 7 6

Step 10:
_ 5 1
4 8 3
2 7 6

Step 11:
4 5 1
_ 8 3
2 7 6

Step 12:
4 5 1
2 8 3
_ 7 6

Step 13:
4 5 1
2 8 3
7 _ 6

Step 14:
4 5 1
2 _ 3
7 8 6

Step 15:
4 _ 1
2 5 3
7 8 6

Step 16:
4 1 _
2 5 3
7 8 6

Step 17:
4 1 3
2 5 _
7 8 6

Step 18:
4 1 3
2 _ 5
7 8 6

Step 19:
4 1 3
_ 2 5
7 8 6

Step 20:
_ 1 3
4 2 5
7 8 6

Step 21:
1 _ 3
4 2 5
7 8 6

Step 22:
1 2 3
4 _ 5
7 8 6

Step 23:
1 2 3
4 5 _
7 8 6

Step 24:
1 2 3
4 5 6
7 8 _

